In [3]:
import json
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# Catastici

In [2]:
# with open('catastici_1741_STD.json', 'r') as f:
#     data = json.load(f)
    
data = gpd.read_file("raw/Catastici_1740_full_geojson_20240603.geojson")

In [3]:
data = pd.DataFrame(data)
print(data.shape)
print(data.columns)
display(data.sample(3))

(33297, 29)
Index(['uidx', 'id', 'owner_name', 'owner_code', 'owner_count',
       'owner_count_remark', 'owner_entity', 'owner_entity_group',
       'owner_first_name', 'owner_family_name', 'owner_family_group',
       'owner_title', 'owner_title_std', 'owner_mestiere',
       'owner_mestiere_std', 'ten_name', 'function', 'an_rendi', 'id_napo',
       'quantity_income', 'quality_income', 'author', 'place', 'parish',
       'sestiere', 'uid', 'path_img', 'quality', 'geometry'],
      dtype='object')


,uidx,id,owner_name,owner_code,owner_count,owner_count_remark,owner_entity,owner_entity_group,owner_first_name,owner_family_name,...,quantity_income,quality_income,author,place,parish,sestiere,uid,path_img,quality,geometry
1237,1237,204,Eredi Giustinian,PPL,2,2+,,,_eredi,GIUSTINIAN,...,nan,nan,Davide,Calle Gioacchina,San Geremia,CN,GER-0204,436_Cannaregio/1_SGeremia/SGeremia_9_188.png,[casa],POINT (12.32375 45.44287)
28868,28868,272,Nobil Homo Trevisan per suo Padre,PPL,1,,,,_padre,TREVISAN,...,nan,nan,Francesca,Corte di Ka' Quartier,San Zulian,SM,ZUL-0272,434_SMarco/5_SZulian/SZulian_12_264.png,[bottega],POINT (12.33961 45.43551)
18366,18366,457,Nobil Homo Ser Francesco Bonfadini,PPL,1,,,,Francesco,BONFADINI,...,nan,nan,Francesca,Corte Bonfadina,San Trovaso,DD,TRV-0457,439_Dorsoduro/6_STrovaso/STrovaso_40_450.png,[casa],POINT (12.32430 45.43142)


In [21]:
data_short = data[['owner_code', 'owner_count',
       'owner_first_name', 'owner_family_name',
       'owner_mestiere', 'ten_name', 'function', 
       'an_rendi', 'place', 'parish', 'quality', 'geometry']]

In [22]:
# numeric rent
# data_short = data_short[data_short['an_rendi'].apply(lambda x: type(x) == int)]
data_short = data_short[data_short['an_rendi'].str.isnumeric()]
data_short.shape

(29169, 12)

In [23]:
# select single owners
# data_short = data_short[data_short['owner_count']==1]
data_short = data_short[data_short['owner_count']=='1']
data_short.shape

(24460, 12)

In [24]:
# Select people only
data_short = data_short[data_short['owner_code']=='PPL']
data_short.shape

(19070, 12)

In [28]:
# clean function
data_short['n_function'] = data_short['quality'].apply(lambda x: len(x))
data_short['function'] = data_short['quality'].apply(lambda x: ', '.join(x))

In [29]:
# remove where | or _ in owner first name
data_short = data_short[~data_short['owner_first_name'].str.contains('\|')]
data_short = data_short[~data_short['owner_first_name'].str.contains('_')]
data_short.shape

(18953, 13)

In [30]:
# clean geometry
data_short['x'] = data_short.geometry.apply(lambda point: point.x)
data_short['y'] = data_short.geometry.apply(lambda point: point.y)
data_short.drop(['geometry', 'owner_count', 'owner_code','quality'], axis=1, inplace=True)

# data_short['geometry'] = data_short['geometry'].apply(lambda x: x.replace('POINT','').replace(')','').replace('(','').strip().split(' '))
# data_short.drop(['owner_count', 'owner_code'], axis=1, inplace=True)

In [31]:
data_short.sample(5)

,owner_first_name,owner_family_name,owner_mestiere,ten_name,function,an_rendi,place,parish,n_function,x,y
30092,Carlo,LAZERONI,,Bortolo Mora,bottega,10,Herbaria Sotto le Fabriche Nove,San Giovanni Elmosinario,1,12.335385,45.439018
23190,Pietro,PERSEGO,,Antonio di Gaetano Gratiosi,"appartamento, casa",28,Sotto il Portico della Tomasini,Santa Croce,2,12.321680,45.436442
18560,Marco,BADOER,,Carlo Milesi,"bottega, casa",42,Corte del Navaro,San Vio,2,12.330816,45.429591
27216,Francesco,BONFADINI,,Francesco Zorzetto,casa,6,Calle di Ca' Da Lezze,San Samuel,1,12.328089,45.434412
2495,Marc'antonio,QUERINI,,Giacomo Valesin,casa,20,Calle de Ca' Querini,San Lunardo,1,12.327065,45.443166


In [232]:
# def convert_coor(geometry):
#     x , y = geometry
#     src_crs = pyproj.CRS('EPSG:32633')
#     tgt_crs = pyproj.CRS('EPSG:4326')
#     transformer = pyproj.Transformer.from_crs(src_crs, tgt_crs, always_xy=True)
#     lon, lat = transformer.transform(x, y)
#     return [lon, lat]
# data_short['geometry'] = data_short['geometry'].apply(convert_coor)

In [32]:
new_columns = {
    "owner_first_name":"Owner_First_Name",
    "owner_family_name":"Owner_Family_Name",
    "owner_mestiere":"Owner_Profession",
    "ten_name":"Tenant_Name",
    "function":"Building_Functions",
    "an_rendi":"Rent_Price",
    "place":"Location",
    "parish":"Parish",
    "x":"Longitude",
    "y":"Latitude",
    "n_function":"Building_Functions_Count",
}
data_short.rename(columns=new_columns, inplace=True)
data_short = data_short.applymap(lambda x: x.lower() if isinstance(x, str) else x)
data_short.sample(5)

,Owner_First_Name,Owner_Family_Name,Owner_Profession,Tenant_Name,Property_Functions,Rent_Price,Location,Parish,Property_Functions_Count,Longitude,Latitude
23747,fabio,bonvicini,,valentin mollin fachin,casetta,10,corte del figher,santa maria mater domini,1,12.330257,45.439539
15688,marco,nespola,,salvator marchesi,casotto,6,riva grande,san gregorio,1,12.329977,45.428664
20901,costantin,loredan,,bortolo caldinelli,casa,60,calle della fortuna,san cassiano,1,12.331910,45.439272
23409,benetto,bruzzi,,simon forni,casa,8,a santa chiara,santa croce,1,12.317419,45.439263
32371,andrea,soranzo,,gabriel noris,bottega,10,campo,san stin,1,12.326615,45.437897


In [34]:
data_short.Owner_Profession.unique()

array(['', 'avvocato', 'medico', 'procurator', 'mercante da legname',
       'orefice', 'illustrissimo medico', 'procuratore', 'fruttarola',
       'luganegher', 'dottor', 'nodaro veneto', 'avocato', 'frutariol',
       'economo', 'mercadante da chiodi', 'cuoco', 'ecconomo',
       'peruchier', 'specier da confetti', "mercante d'oro", 'tutrice',
       'erbarol', 'fabro'], dtype=object)

In [35]:
data_short = data_short.fillna('')
data_short.to_csv('clean/buildings_1740.csv', index=False)

In [36]:
data_short = pd.read_csv('clean/buildings_1740.csv')

In [37]:
gdf = gpd.GeoDataFrame(data_short, geometry=[Point(xy) for xy in zip(data_short['Longitude'], data_short['Latitude'])])
geojson_str = gdf.to_json()
gdf.to_file("clean/buildings_1740_geo.geojson", driver="GeoJSON")  

# OSM

In [39]:
with open('raw/export.geojson', 'r') as f:
    data = json.load(f)['features']
data_p = [d['properties'] for d in data]
data_g = [d['geometry']['coordinates'] for d in data]

In [40]:
all_keys = set([dk for d in data_p for dk in d.keys()])
print(all_keys)

{'name:ru', 'roof:shape', 'name:en', 'name:pl', 'name:uk', 'service_times', 'type', 'name', 'svg:bezier', 'roof:colour', 'name:zh', 'old_name', 'lit', 'horse', 'roof:material', 'name:da', 'style', 'wikipedia:en', 'note', 'foot', 'name:lt', 'addr:city', 'religion', 'name:fr', 'addr:postcode', 'roof:orientation', 'bicycle', 'wikidata', 'source', 'area', 'importance', 'name:hu', 'check_date', 'addr:street', 'name:ko', 'start_date', '@id', 'wheelchair', 'building', 'name:pt', 'surface', 'addr:housenumber', 'highway', 'visible_name', 'layer', '@geometry', 'building:colour', 'addr:place', 'name:vec', 'email', 'url', 'loc_name', 'name:he', 'name:sk', 'place', 'name:cs', 'building:levels', 'website', 'source:date', 'amenity', 'wikipedia', 'abandoned', 'wikimedia_commons', 'name:es', 'opening_hours', 'roof:height', 'name:de', 'building:material', 'ele', 'name:it', 'height', 'name:ja', 'tourism', 'addr:country', 'alt_name', 'historic', 'denomination'}


In [41]:
important_keys = ['name','religion','building','amenity','highway','place']
data_clean = []
for p, g in zip(data_p, data_g):
    p_k = {}
    for k in important_keys:
        if k in p.keys():
            p_k[k] = p[k]
    p_k['geometry'] = g
    data_clean.append(p_k)
    

In [42]:
data_exp = pd.DataFrame(data_clean)
data_exp.sample(5)

,religion,building,amenity,geometry,name,highway,place
196,jewish,synagogue,place_of_worship,"[12.3266278, 45.4451422]",Scuola Italiana,NaN,NaN
15,NaN,NaN,NaN,"[12.3278968, 45.433487]",Campo San Samuele,pedestrian,square
87,christian,yes,place_of_worship,"[12.3363048, 45.4401811]",Scuola dell'Angelo custode,NaN,NaN
116,christian,church,place_of_worship,"[12.3410016, 45.4370337]",Chiesa di Santa Maria Formosa,NaN,NaN
86,christian,church,place_of_worship,"[12.3598627, 45.4345413]",Basilica di San Pietro di Castello,NaN,NaN


In [43]:
data_exp.loc[(data_exp['highway'].notna()) | (data_exp['place'].notna()), 'type'] = 'square'
data_exp.loc[(data_exp['building'].notna()), 'type'] = 'building'
data_exp = data_exp[data_exp['religion'] != 'jewish']
data_exp.loc[data_exp['name']=='Oratorio della Madre di Dio', 'building'] = 'church'
data_exp.loc[data_exp['name']=='Chiesa di San Giovanni Battista ai Catecumeni', 'building'] = 'church'
data_exp.loc[data_exp['name']=='Cappella di San Vio', 'building'] = 'church'
data_exp.loc[data_exp['name']=="Scuola dell'Angelo custode", 'building'] = 'church'
data_exp.loc[data_exp['building']=="chapel", 'building'] = 'church'
data_exp.loc[data_exp['building'].notna(), 'type'] = data_exp['building']
data_exp['name'].fillna('',inplace=True)
data_exp['Longitude'] = data_exp['geometry'].apply(lambda x: x[0])
data_exp['Latitude'] = data_exp['geometry'].apply(lambda x: x[1])
data_exp.drop(['geometry','highway','place','religion','building','amenity'], axis=1, inplace=True)
data_exp['name'] = data_exp['name'].apply(lambda x: x.lower())
data_exp.rename(columns={"name":"Entity_Name", "type":"Entity_Type"}, inplace=True)

In [44]:
print(data_exp.shape)
data_exp.sample(5)

(192, 4)


,Entity_Name,Entity_Type,Longitude,Latitude
35,corte del tintor,square,12.337313,45.438169
115,san giovanni grisostomo,church,12.337255,45.439156
175,chiesa di san gallo,church,12.336870,45.434556
158,basilica dei santi giovanni e paolo,church,12.342197,45.439254
150,chiesa di san giacomo dall'orio,church,12.327433,45.440122


In [45]:
data_exp.to_csv('clean/landmarks.csv', index=False)

In [46]:
data_exp = pd.read_csv('clean/landmarks.csv')

gdf = gpd.GeoDataFrame(data_exp, geometry=[Point(xy) for xy in zip(data_exp['Longitude'], data_exp['Latitude'])])
geojson_str = gdf.to_json()
gdf.to_file("clean/landmarks_geo.geojson", driver="GeoJSON")  
print(geojson_str)

{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {"Entity_Name": null, "Entity_Type": "church", "Longitude": 12.3328817, "Latitude": 45.4463708}, "geometry": {"type": "Point", "coordinates": [12.3328817, 45.4463708]}}, {"id": "1", "type": "Feature", "properties": {"Entity_Name": "campiello della malvasia", "Entity_Type": "square", "Longitude": 12.3497984, "Latitude": 45.4329065}, "geometry": {"type": "Point", "coordinates": [12.3497984, 45.4329065]}}, {"id": "2", "type": "Feature", "properties": {"Entity_Name": "campo santa margherita", "Entity_Type": "square", "Longitude": 12.3234482, "Latitude": 45.434305}, "geometry": {"type": "Point", "coordinates": [12.3234482, 45.434305]}}, {"id": "3", "type": "Feature", "properties": {"Entity_Name": "campo sant'anzolo", "Entity_Type": "square", "Longitude": 12.3319525, "Latitude": 45.4342152}, "geometry": {"type": "Point", "coordinates": [12.3319525, 45.4342152]}}, {"id": "4", "type": "Feature", "properties

# Sommaroni

In [4]:
data = gpd.read_file("raw/sommarioni_geometries_function_and_people_standardized_20240423.geojson")
    
data = pd.DataFrame(data)
print(data.shape)
print(data.columns)
display(data.sample(3))

(39742, 30)
Index(['id', 'parcel_number', 'parcel_type', 'geometry_id', 'unique_id',
       'district_acronym', 'sub_parcel_number',
       'austrian_cadaster_correspondance',
       'austro_italian_cadaster_correspondance', 'place', 'house_number',
       'owner', 'owner_standardised', 'quality', 'area', 'ownership_types',
       'qualities', 'own_uid', 'own_nucleus_uid', 'own_family', 'own_name',
       'own_title', 'own_father', 'own_father_is_q', 'own_mother',
       'own_siblings', 'own_husband', 'own_husband_is_q', 'own_other_notes',
       'geometry'],
      dtype='object')


,id,parcel_number,parcel_type,geometry_id,unique_id,district_acronym,sub_parcel_number,austrian_cadaster_correspondance,austro_italian_cadaster_correspondance,place,...,own_name,own_title,own_father,own_father_is_q,own_mother,own_siblings,own_husband,own_husband_is_q,own_other_notes,geometry
8837,way/18388,12788,building,3102,19396.0,NDD,1,432,432,DD - Fondamenta Malcanton Calle della Biaca,...,Angelo Samuele,,,NaN,,,,NaN,,"POLYGON ((12.32237 45.43629, 12.32248 45.43627..."
36168,way/13771,9104,building,14741,13673.0,NSP,None,ASP 800,AISP 800,SP - Ruga degli Orefici,...,Giovanni Francesco,,RASPI Giovanni Maria,1.0,,,,NaN,,"POLYGON ((12.33519 45.43847, 12.33521 45.43848..."
35411,way/31585,8702,building,14294,13207.0,NCS,None,3190,3190,"CS - Strada Eugenia, Corte Friziera, Fondament...",...,Giuseppe,,SCARPA Giacomo,1.0,,,,NaN,,"POLYGON ((12.35543 45.43265, 12.35534 45.43265..."


In [5]:
data = data[['parcel_type', 'district_acronym',
       'place', 'area', 'ownership_types',
       'qualities', 'own_uid', 'own_family', 'own_name',
       'geometry']]

In [6]:
# keep only building
data = data[(data['parcel_type'] == 'building')]
data = data.drop('parcel_type', axis=1)

In [7]:
# filter notna owners
data = data[data['own_uid'].notna()]
data = data.drop('own_uid', axis=1)

In [8]:
# filter given ownerships
data = data[data['ownership_types'].apply(lambda x: len(x))!=0]
data = data.drop('ownership_types', axis=1)

In [9]:
# rename districts
districts = {'NSM':'San Marco', 'NCS':'Castello', 'NCN':'Cannaregio', 'NSP':'San Polo', 'NSC':'Santa Croce', 'NDD':'Dorsoduro'}
data = data[data['district_acronym']!='NCC']
data['district_acronym'] = data['district_acronym'].map(districts)

In [10]:
# clean place
data = data[data['place'].notna()]
data['place'] = data['place'].apply(lambda x: x.replace('–', '-'))
data['place'] = data['place'].apply(lambda x: x.split('-')[1].split(',')[0].strip())

In [11]:
# final preprocessing
data['Building_Functions_Count'] = data['qualities'].apply(lambda x: len(x))
data['Building_Functions'] = data['qualities'].apply(lambda x: ', '.join(x))
data.drop('qualities', axis=1, inplace=True)

data.rename({'place':'Location'}, axis=1, inplace=True)
data.rename({'area':'Building_Area'}, axis=1, inplace=True)
data.rename({'own_family':'Owner_Family_Name'}, axis=1, inplace=True)
data.rename({'own_name':'Owner_First_Name'}, axis=1, inplace=True)
data.rename({'district_acronym':'District'}, axis=1, inplace=True)

data['geometry'] = data['geometry'].apply(lambda x: x.centroid)
data['Longitude'] = data['geometry'].apply(lambda x: x.x)
data['Latitude'] = data['geometry'].apply(lambda x: x.y)
data.drop('geometry', axis=1, inplace=True)

for col in ['Building_Functions','District','Location','Owner_Family_Name','Owner_First_Name']:
    data[col] = data[col].apply(lambda x: x.lower())

In [12]:
data.sample(5)

,District,Location,Building_Area,Owner_Family_Name,Owner_First_Name,Property_Functions_Count,Property_Functions,Longitude,Latitude
33512,cannaregio,sottoportico e corte giustiniana,79.809,codognola,gerolamo,1,casa,12.319801,45.446740
25637,castello,ramo della pegola,479.665,fini,vincenzo,2,"casa, corto",12.349668,45.433947
19590,castello,calle e fondamenta del rimedio,908.568,tana,andrea,2,"casa, magazzeno",12.340767,45.436234
16484,castello,calle di san domenico,625.801,marchesani,giorgio,2,"corto, magazzeno",12.354202,45.430805
9286,dorsoduro,fondamenta ponte rosso,625.470,canal,antonia,2,"casa, corto",12.320871,45.434277


In [13]:
data = data.reset_index(drop=True)
data.to_csv('clean/buildings_1808.csv', index=False)

gdf = gpd.GeoDataFrame(data, geometry=[Point(xy) for xy in zip(data['Longitude'], data['Latitude'])])
geojson_str = gdf.to_json()
gdf.to_file("clean/buildings_1808_geo.geojson", driver="GeoJSON")  